In [ ]:
!pip install gdown > /dev/null
!apt install unrar > /dev/null
import gdown
url = 'http://drive.google.com/uc?export=view&id=1VDK9um994V-z5D1d6hxOOyQ0G3ShpaMH'
output = './data.rar'
gdown.download(url, output, quiet=False)
!unrar x data.rar ./ > /dev/null
url = 'http://drive.google.com/uc?export=view&id=1lgOkF8M7DfPl2fWHUvXCCQZykXISexKv'
output = './annot.rar'
gdown.download(url, output, quiet=False)
!unrar x annot.rar ./ > /dev/null

In [ ]:
import xml.etree.ElementTree as et
import os
import shutil
import glob
from sklearn.model_selection import train_test_split
xml_path = './Vehicle Plates annotations/'
txt_path = './vehicle_plate_annot_txt/'
!mkdir ./vehicle_plate_annot_txt/

def xml_to_txt(path, dest):
    tree = et.parse(path)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    f = open(dest, 'w')
    results = []
    for obj in root.findall('object'):
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        pil_bbox[0], pil_bbox[1], pil_bbox[2], pil_bbox[3] = (pil_bbox[2] + pil_bbox[0]) / (2 * width), \
        (pil_bbox[3] + pil_bbox[1]) / (2 * height), (pil_bbox[2] - pil_bbox[0]) / width, (pil_bbox[3] - pil_bbox[1]) / height
        results.append(' '.join(['0'] + [str(x) for x in pil_bbox]))
    f.write('\n'.join(results))
    f.close()

file_names = os.listdir(xml_path)
fail = []
for file in file_names:
    path = xml_path + file
    dest = txt_path + file[:-3] + 'txt'
    try:
        xml_to_txt(path, dest)
    except:
        fail.append(file)

In [ ]:
org_path = './Vehicle Plates/'
org_lbl = './vehicle_plate_annot_txt/'
train_path = './dataset/images/train/'
valid_path = './dataset/images/valid/'
train_lbl = './dataset/labels/train/'
valid_lbl = './dataset/labels/valid/'

!mkdir -p ./dataset/images/train/
!mkdir -p ./dataset/images/valid/
!mkdir -p ./dataset/labels/train/
!mkdir -p ./dataset/labels/valid/

file_names = os.listdir(org_path)
train, valid = train_test_split(file_names, test_size = 0.2, random_state = 8)

for file in train:
    src = org_path + file
    dst = train_path + file
    shutil.copyfile(src, dst)
for file in valid:
    src = org_path + file
    dst = valid_path + file
    shutil.copyfile(src, dst)

for file in train:
    src = org_lbl + file[:-3] + 'txt'
    dst = train_lbl + file[:-3] + 'txt'
    shutil.copyfile(src, dst)
for file in valid:
    src = org_lbl + file[:-3] + 'txt'
    dst = valid_lbl + file[:-3] + 'txt'
    shutil.copyfile(src, dst)

In [ ]:
import yaml

data = {'path':'../dataset' ,'train':'images/train', 'val':'images/valid', 'nc': 1, 'names': ['plate']}
with open('./data.yaml', 'w') as f:
    yaml.dump(data, f, default_flow_style = True)

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

In [ ]:
!python train.py --img 224 --batch 8 --epoch 100 --data /content/data.yaml --weights yolov5s.pt --project plate-detection

train: weights=yolov5s.pt, cfg=, data=/content/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=8, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=plate-detection, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-53-gf0e5a60 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, 